In [1]:
import sys
import time
import datetime
import os
import tensorflow as tf
import argparse
import json
import socket
from PIL import Image
from util import *
import numpy as np
%load_ext autoreload
%autoreload 2
# %matplotlib notebook
from copy import deepcopy
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from matplotlib.colors import hsv_to_rgb
from helpers import *

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
logPath = "TRAINING/split/train/"
snapshotPath = "SNAPSHOTS/snapshots/"

In [3]:
with open("hyperParams.json") as f:
    instanceParams = json.load(f)

# instanceParams

In [4]:
printFrequency = instanceParams["printFreq"]
batchSize = instanceParams["batchSize"]

iterations = instanceParams["iterations"]
baseLearningRate = instanceParams["baseLR"]
learningRate = baseLearningRate
snapshotFrequency = instanceParams["snapshotFreq"]

from dotmap import DotMap
arg = DotMap()
arg.logDev = False
arg.resume = 'n'
resume, startIteration, snapshotFiles = checkResume(snapshotPath,logPath, arg)

No snapshots found, training from scratch


In [5]:
with tf.device("/gpu:0"):
    trainingData = TrainingData(batchSize,instanceParams)

Reading Tensor("ImagePairData/image_data_reader/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_1/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_2/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_3/Placeholder:0", dtype=string, device=/device:GPU:0)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
with tf.device("/gpu:0"):
    flow_gd = tf.Variable(tf.zeros([2, 448, 832, 2]), name = "my_new_var")
    photoAlpha = instanceParams["photoParams"]["robustness"]
    photoBeta = instanceParams["photoParams"]["scale"]
    gamma = 1.0e5
    sd_lr = 1.0

In [7]:
with tf.device("/gpu:0"):
    # init
    with tf.variable_scope("netShare"):
        networkBodyF = NetworkBody(trainingData,instanceParams)
    with tf.variable_scope("netShare",reuse=True):
        networkBodyB = NetworkBody(trainingData,instanceParams,flipInput=True)

    trainingLoss = TrainingLoss(instanceParams,networkBodyF,networkBodyB,trainingData, flow_gd)
    solver, solver_op, learningRateTensor = attachSolver(trainingLoss.loss)
    flowFinal = networkBodyF.flows[0]
    # loss scheduling
    recLossBWeightTensor = trainingLoss.recLossBWeight

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [8]:
with tf.device("/gpu:0"):
    ploss = photoLoss(flow_gd,
                      trainingData.frame0["rgbNorm"],
                      trainingData.frame1["rgbNorm"],
                      photoAlpha, photoBeta)
    closs = tf.reduce_sum(tf.square(flow_gd - tf.stop_gradient(flowFinal)), axis=3, keep_dims=True) * gamma
    tf.summary.image("fhat", flowToRgb1(flow_gd))
    tf.summary.scalar("fhat_ploss", tf.reduce_mean(ploss))

    with tf.variable_scope(None, default_name="new_var_solver"):
        split_solver = tf.train.AdamOptimizer(learning_rate=sd_lr, beta1=0.9, beta2=0.999)
        split_solver_op = split_solver.minimize(closs + ploss)

In [9]:
variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

var_subset = []
uninit = []
for v in variables:
    if not ("new_var" in v.name):
        var_subset.append(v)
    else:
        uninit.append(v)

In [10]:
# merge summaries
merged = tf.summary.merge_all()
saver = tf.train.Saver(var_subset, max_to_keep=0)

In [11]:
printFrequency = 100
snapshotFrequency = 2000
iterations = 20000
startIteration = 0
print(printFrequency)
print(snapshotFrequency)
print(iterations)
print(startIteration)

100
2000
20000
0


In [ ]:
fs = []
fhats = []
with sessionSetup(arg) as sess:
    # sess.run(tf.initialize_all_variables())
    sess.run(tf.initialize_variables(uninit))
    saver.restore(sess, '../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt')
    trainingData.dataQueuer.start_queueing(sess)

    #start summary writer
    summary_writer = tf.summary.FileWriter(logPath, sess.graph)

    #run
    lastPrint = time.time()
    for i in range(startIteration, startIteration + iterations + 10):
        # scheduled values
        learningRate = learningRateSchedule(baseLearningRate, i)
        recLossBWeight = unsupLossBSchedule(i)

         #run training
        feed_dict = {
            learningRateTensor: 1e-5
        }
        summary,result,totalLoss, f1, nfg, pl, f2 = sess.run([
            merged,solver_op,trainingLoss.loss, flowFinal, split_solver_op, ploss, flow_gd], feed_dict=feed_dict)

        if (i+1) % printFrequency == 0:
            timeDiff = time.time() - lastPrint
            itPerSec = printFrequency/timeDiff
            remainingIt = startIteration + iterations + 10 - i
            eta = remainingIt/itPerSec
            print("Iteration "+str(i+1)+": loss: "+str(totalLoss)+", iterations per second: "+str(
                itPerSec)+", ETA: "+str(datetime.timedelta(seconds=eta)))+", lr: "+str(learningRate)

            summary_writer.add_summary(summary,i+1)
            summary_writer.flush()
            lastPrint = time.time()
            fs.append(deepcopy(f1))
            fhats.append(deepcopy(f2))

        if (i+1) % snapshotFrequency == 0:
            saver.save(sess,"snapshots/iter_"+str(i+1).zfill(16)+".ckpt")

        sys.stdout.flush()

    trainingData.dataQueuer.close(sess)

Instructions for updating:
Use `tf.variables_initializer` instead.
INFO:tensorflow:Restoring parameters from ../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt
Iteration 100: loss: [[620.3993]
 [620.3994]], iterations per second: 1.98998492399, ETA: 2:46:45.603439, lr: 1e-05
Iteration 200: loss: [[563.73114]
 [563.73175]], iterations per second: 2.16363849045, ETA: 2:32:36.335537, lr: 1e-05
Iteration 300: loss: [[553.9345]
 [553.9348]], iterations per second: 2.06024909632, ETA: 2:39:27.289720, lr: 1e-05
Iteration 400: loss: [[554.48804]
 [554.48816]], iterations per second: 1.89403298579, ETA: 2:52:34.096337, lr: 1e-05
Iteration 500: loss: [[552.6535 ]
 [552.65643]], iterations per second: 1.8123960608, ETA: 2:59:25.307000, lr: 1e-05
Iteration 600: loss: [[544.84424]
 [544.84656]], iterations per second: 1.77703136007, ETA: 3:02:03.273745, lr: 1e-05
Iteration 700: loss: [[534.6983]
 [534.692 ]], iterations per second: 1.76456147933, ETA: 3:02:23.795513

Iteration 6400: loss: [[353.6892 ]
 [353.69485]], iterations per second: 1.62073708694, ETA: 2:19:58.030816, lr: 1e-05
Iteration 6500: loss: [[334.76968]
 [334.79156]], iterations per second: 1.61924796518, ETA: 2:19:03.996899, lr: 1e-05
Iteration 6600: loss: [[331.51038]
 [331.50012]], iterations per second: 1.6230197199, ETA: 2:17:42.992640, lr: 1e-05
Iteration 6700: loss: [[351.90277]
 [352.00122]], iterations per second: 1.62191634382, ETA: 2:16:46.958423, lr: 1e-05
Iteration 6800: loss: [[332.13644]
 [332.09222]], iterations per second: 1.63024437139, ETA: 2:15:03.693061, lr: 1e-05
Iteration 6900: loss: [[332.81067]
 [332.82037]], iterations per second: 1.63176306507, ETA: 2:13:54.867488, lr: 1e-05
Iteration 7000: loss: [[335.66333]
 [335.6905 ]], iterations per second: 1.63423283395, ETA: 2:12:41.533834, lr: 1e-05
Iteration 7100: loss: [[335.77927]
 [335.77695]], iterations per second: 1.64617079425, ETA: 2:10:43.050092, lr: 1e-05
Iteration 7200: loss: [[356.93234]
 [357.19824]],

Iteration 13000: loss: [[289.08395]
 [289.12177]], iterations per second: 1.69548611683, ETA: 1:08:55.097262, lr: 1e-05
Iteration 13100: loss: [[300.95087]
 [300.9523 ]], iterations per second: 1.69621138972, ETA: 1:07:54.374245, lr: 1e-05
Iteration 13200: loss: [[307.74078]
 [307.76773]], iterations per second: 1.70272461254, ETA: 1:06:40.059640, lr: 1e-05
Iteration 13300: loss: [[291.85095]
 [291.85165]], iterations per second: 1.7007793445, ETA: 1:05:45.838137, lr: 1e-05
Iteration 13400: loss: [[292.58252]
 [292.60733]], iterations per second: 1.69933051798, ETA: 1:04:50.355602, lr: 1e-05
Iteration 13500: loss: [[488.74664]
 [488.73334]], iterations per second: 1.70027518585, ETA: 1:03:49.380123, lr: 1e-05
Iteration 13600: loss: [[346.8853 ]
 [346.89764]], iterations per second: 1.70169014391, ETA: 1:02:47.430882, lr: 1e-05
Iteration 13700: loss: [[617.07666]
 [617.2301 ]], iterations per second: 1.70226772178, ETA: 1:01:47.407430, lr: 1e-05
Iteration 13800: loss: [[352.76062]
 [352

### Testing

In [ ]:
datasetRoot = '../example_data/'
frame0Path = '../example_data/datalists/train_im0.txt'
frame1Path = '../example_data/datalists/train_im1.txt'
gt0Path = '../example_data/datalists/train_gt0.txt'
desiredHeight = 480
desiredWidth = 854

In [ ]:
with open(frame0Path) as f:
    imagePairs0 = [datasetRoot+x[:-1] for x in f.readlines()]
with open(frame1Path) as f:
    imagePairs1 = [datasetRoot+x[:-1] for x in f.readlines()]
with open(gt0Path) as f:
    gtPairs1 = [datasetRoot+x[:-1] for x in f.readlines()]

In [ ]:
imagePairs0 = [item for item in imagePairs0 if 'bus' in item]
imagePairs1 = [item for item in imagePairs1 if 'bus' in item]
gtPairs1 = [item for item in gtPairs1 if 'bus' in item]

In [ ]:
testData = TestData(imagePairs0,imagePairs1,gtPairs1,1,desiredHeight,desiredWidth)

In [ ]:
with tf.device("/gpu:0"):
    with tf.variable_scope("netShare"):
        networkBody = NetworkBody(testData,instanceParams)
    flowFinal = networkBody.flows[0]

In [ ]:
flowViz = flowToRgb(flowFinal)
transformGrid = flowTransformGrid(flowFinal)
mean = tf.expand_dims(tf.expand_dims(tf.expand_dims([0.407871, 0.457525, 0.481094], 0), 0), 0)

In [ ]:
saver = tf.train.Saver()

# config tensorflow
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True

In [ ]:
test_length = 70
start = 0
result_dir = 'results'
iterations = 515000

flows = []
viz = []
gradients = []
ground_truths = []
arrs = []
with tf.Session(config=config) as sess:
    saver.restore(sess,"snapshots/iter_0000000000002000.ckpt")

    # run
    lastPrint = time.time()
    for i in range(start, start + min(test_length, len(imagePairs0))):
        feed_dict = {
            testData.im0File: imagePairs0[i],
            testData.im1File: imagePairs1[i],
            testData.gt0File: gtPairs1[i]
        }
        hsv, f = sess.run([flowViz, flowFinal],feed_dict=feed_dict)

        flows.append(deepcopy(f))
        h, w = 448, 532
        arr = np.maximum(np.minimum(np.asarray(hsv),1), 0)
        arr = np.squeeze(np.asarray(arr*255,np.uint8))
        arrs.append(deepcopy(arr[:h,:w,:]))
        im = Image.fromarray(arr[:h,:w,:])
        viz.append(deepcopy(im))
        # im.save("{}/{}.png".format(result_dir, str(i).zfill(3)))